In [1]:
# https://pypi.org/project/torch-geometric/
# pip install torch
# pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
# pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
# pip install torch-geometric


# https://towardsdatascience.com/louvains-algorithm-for-community-detection-in-python-95ff7f675306
import torch
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import Counter 

In [2]:
# plot the data
from utils.visual import plot

#plot(0) 

In [3]:
import re

def find_train_ego_index():
    pre, pree = [], []
    for item in train.files:
        pre.append(int(re.sub('\D', '',item )))

    for i in egonet.files:
        pree.append(int(re.sub('\D', '',i )))
        
    train_ego_index = []
    for i in range(len(pree)):
        if pree[i] in pre:
            train_ego_index.append(i)
    return train_ego_index, pree, pre

In [4]:
# self-made packages
from utils.data import data_preprocess


train = data_preprocess("\Training")
train_list, train_dict = train.train_data()
df = pd.DataFrame(data=train_dict)

In [5]:
egonet = data_preprocess("\egonets")
ego_list, user_list = egonet.ego_data()

In [6]:
# 這個是指應該將哪些 ego_list 用在 training，方便輸入0,1,2，就不用輸入 239、345、661...
train_ego_index,ego_user, train_user  = find_train_ego_index()

## Data Preparation

In [7]:
from utils.user import users

users = users()
index = user_list[train_ego_index[2]] # 更改數字可以挑選要哪一個 training data 

# edge information EX: (240, 241), (240, 242)...
edge1 = egonet.edge_data(train_ego_index[0])

In [8]:
#users = pd.read_csv('df_new_friend.csv') #把註解拿掉代表讀取新的feature

## Predict Data

In [9]:
circlesP = {}

# 這裡是整理所有的 predict data，但因為目前沒有所以暫時用 training 的代替
for i in range(len(train_dict)):
    uidP,friendsP = train_dict[i]['UserId'], train_dict[i]['Predicted'] # predict 
    circlesP[int(uidP)] = [set([int(x) for x in c.split()]) for c in friendsP.split(';')]

## Connected Component
#### 測試connected component

In [10]:
from connected_components import findCommunities
import sys, os

def connected_componet(target_dir):
    egoUser = -1
    try:
        egoUser = int(re.sub("\D", '', target_dir))
    except Exception as e:
        print("Expected files to be names 'X.egonet' where X is a user ID")

    cs = list(findCommunities(target_dir))

    if len(cs) == 0:
        cs = [set(adj.keys())]
    cs = [' '.join([str(y) for y in x]) for x in cs]
    res = str(egoUser) + ',' + ';'.join(cs)
    
    return res
    

In [11]:
def process(test_ego_num):
    return [{
            'UserId': test_ego_num.split(',')[0],
            'Predicted': test_ego_num.split(',')[1]
            }]


In [12]:
connect_egonet = []
for i in range(len(train.files)):
    target_dir = os.getcwd()+"\\egonets\\"+train.files[i].replace('circles', 'egonet')
    test_ego_num = connected_componet(target_dir)
    test = process(test_ego_num)
    connect_egonet+=(test)

# Caculate Loss
### 之後可能要放進一個mode裡面當作 objective function 之類的，這邊先暫時表達一次性的評分

## Model

In [13]:
from sklearn.cluster import KMeans

def exe(n_clusters):
    km = KMeans(
        n_clusters=n_clusters, init='random',
        n_init=10, max_iter=300, 
        tol=1e-04, random_state=2
    )
    final, indexx = [], []
    for i in range(len(train_ego_index)):
        index = user_list[train_ego_index[i]] # 更改數字可以挑選要哪一個 training data 
        test = users.iloc[index]

        X = np.array(test).astype(np.int64)
        return X
        y_km = km.fit(X)
        predict = y_km.predict(X)
        final.append(predict)
        indexx.append(index)
    return final, indexx

In [14]:
from socialCircles_metric import loss1

def temperal(train_dict):
        
    circlesG = {}

    for i in range(len(train_dict)):
        uidG,friendsG = train_dict[i]['UserId'], train_dict[i]['Predicted'] # ground truth
        circlesG[int(uidG)] = [set([int(x) for x in c.split()]) for c in friendsG.split(';')] 
        
        
    totalLoss = 0
    for k in circlesP.keys():
        try:
            if k not in circlesG:
                print("Ground-truth has prediction for circle", k, "but prediction does not")
                break

            l = loss1(circlesG[k], circlesP[k])
            #print("loss for user", k, "=", l)
            totalLoss += l
        except:
            continue
            
    print("total loss for all users =", totalLoss)

In [15]:
def data(n_clusters):

    final, index = exe(n_clusters)
    
    resss = []
    for k in range(len(final)):
        fina = ""
        for i in range(n_clusters):
            tem = ''

            for j in range(len(final[k])):
                if final[k][j] == i:
                    tem+= ' ' + str(index[k][j])
            fina += tem.strip() + ';'
        resss.append(fina)
        
    transform_label = []
    
    for i in range(len(index)):
        transform_label.append({"UserId":index[i][0]-1, "Predicted":resss[i].strip()[:-1]})
        
    return transform_label

In [16]:
temperal(connect_egonet)

total loss for all users = 17625


In [17]:
temperal(train_dict)

total loss for all users = 0


In [18]:
index = train_ego_index[30]
first_user = user_list[index][0]

In [19]:
from collections import Counter

def count_similarity(a, b):
    count = 0
    for i in range(len(a)):
        if (a[i] != -1) and (a[i] == b[i]):
            count+=1
            
    return count/(len(a) - Counter(a)[-1]) # 分母為去除掉 -1 的個數

In [20]:
test = users.iloc[user_list[index]]

In [21]:
res = []
for j in range(len(user_list[index])):
    user, friend = ego_list[index][j][0], ego_list[index][j][1:]
    temp1 = []
    for i in range(len(friend)):
        temp1.append(count_similarity(test.iloc[user-first_user], test.iloc[friend[i] - first_user]))
    res.append(temp1)

In [22]:
res

[[0.11764705882352941,
  0.058823529411764705,
  0.058823529411764705,
  0.11764705882352941],
 [0.0, 0.2, 0.1, 0.0],
 [0.0],
 [0.1, 0.2, 0.1, 0.1, 0.2, 0.2, 0.2, 0.1, 0.3, 0.2, 0.2],
 [0.0,
  0.0,
  0.0,
  0.07692307692307693,
  0.07692307692307693,
  0.07692307692307693,
  0.07692307692307693,
  0.07692307692307693,
  0.0,
  0.15384615384615385,
  0.07692307692307693,
  0.07692307692307693,
  0.07692307692307693,
  0.07692307692307693,
  0.0,
  0.15384615384615385,
  0.07692307692307693,
  0.15384615384615385,
  0.07692307692307693,
  0.0,
  0.15384615384615385,
  0.07692307692307693,
  0.07692307692307693,
  0.15384615384615385,
  0.07692307692307693],
 [0.0, 0.0, 0.125, 0.1875, 0.125, 0.1875],
 [0.1111111111111111,
  0.1111111111111111,
  0.2222222222222222,
  0.1111111111111111],
 [0.125, 0.125, 0.125, 0.125],
 [0.1875],
 [],
 [0.3333333333333333],
 [0.16666666666666666, 0.0, 0.16666666666666666],
 [0.14285714285714285, 0.14285714285714285, 0.14285714285714285],
 [0.0625, 0.0625, 

In [23]:
ego_list[index]

[[11365, 11406, 11372, 11371, 11388],
 [11366, 11391, 11393, 11376, 11392],
 [11367, 11391],
 [11368,
  11381,
  11401,
  11389,
  11394,
  11369,
  11370,
  11404,
  11396,
  11405,
  11402,
  11383],
 [11369,
  11381,
  11389,
  11391,
  11385,
  11394,
  11386,
  11408,
  11380,
  11392,
  11400,
  11406,
  11378,
  11395,
  11372,
  11384,
  11404,
  11371,
  11368,
  11407,
  11382,
  11402,
  11379,
  11397,
  11390,
  11388],
 [11370, 11381, 11389, 11394, 11404, 11368, 11402],
 [11371, 11369, 11406, 11378, 11365],
 [11372, 11369, 11406, 11365, 11388],
 [11373, 11404],
 [11374],
 [11375, 11384],
 [11376, 11366, 11392, 11377],
 [11377, 11393, 11376, 11392],
 [11378, 11391, 11369, 11406, 11371, 11405],
 [11379, 11386, 11408, 11369, 11404],
 [11380, 11369],
 [11381,
  11401,
  11389,
  11394,
  11369,
  11395,
  11370,
  11404,
  11368,
  11396,
  11405,
  11402],
 [11382, 11391, 11369, 11384, 11407],
 [11383, 11389, 11387, 11386, 11400, 11404, 11368, 11402, 11397, 11390],
 [11384, 

In [26]:
color_map = []
node_groups = []

for i in range(len(train_list[0])):
    node_groups.append(list(map(int, train_list[30][i])))

for node in G:
    temp = len(color_map)
    for i in range(len(node_groups)):
        if node in node_groups[i]:
            color_map.append(colors[i])
            break

    # if no element add, then the node is not in any list         
    if temp == len(color_map):
        color_map.append("red")

nx.draw(G, node_color=color_map, with_labels=True)
plt.show()

NameError: name 'G' is not defined

In [ ]:
import community
from utils.visual import plot, draw, colors
import networkx as nx
import networkx.algorithms.community as nx_comm

# Create a graph using NetworkX
def do():
    aa=[]
    bb={}
    
    for i in range(len(train_ego_index)):
        index = train_ego_index[i]
        first_user = user_list[index][0]
        temp_path = os.path.join(egonet.target_dir, egonet.files[index]) #51
        G = draw(temp_path)
        
        bb[first_user-1] = nx_comm.louvain_communities(G,threshold=0.1 , seed=123)

#         partition = community.best_partition(G)
#         res = sorted(partition.items(), key=lambda item: item[1])
#         aa+=process_res(res, index)
    return bb #aa

In [ ]:
def process_res(res, first_user):
    temp = ''
    index = 0
    a = ''
    for i in range(len(res)):
        if index != res[i][1]:
            index = res[i][1]
            temp += a.strip() + ';'
            a = ''
        a += ' '+str(res[i][0])
    temp += a.strip()
        
    return [{'UserId':first_user-1, 'Predicted':temp}]

In [ ]:
aaa = do()

In [ ]:
from socialCircles_metric import loss1

def another_temperal(circlesG):
        
    totalLoss = 0
    for k in circlesP.keys():
        try:
            if k not in circlesG:
                print("Ground-truth has prediction for circle", k, "but prediction does not")
                #break

            l = loss1(circlesG[k], circlesP[k])
            #print("loss for user", k, "=", l)
            totalLoss += l
        except:
            continue
            
    print("total loss for all users =", totalLoss)

In [ ]:
another_temperal(aaa)